In [ ]:
%load_ext autoreload
%autoreload 2
from ASD_Circuits import *

# Old

In [ ]:
g = LoadConnectome2(ConnFil=
                    "../dat/allen-mouse-conn/norm_density-max_ipsi_contra-pval_0.05-deg_min_1-by_weight_pvalue.csv")
EdgeWeightsDict = EdgeDict(g, keyon="label")
ASD_Bias = pd.read_csv("dat/Jon_data/Spark_Final_ExpBias.csv", index_col="STR")

In [ ]:
N_sim = 1000
weighted, directed = False, False
asd_vs_rand = TopNAndCohesiveness(ASD_Bias, g, weighted, directed, EdgeWeightsDict, "rand", N_sim)
asd_vs_sib = TopNAndCohesiveness(ASD_Bias, g, weighted, directed, EdgeWeightsDict, "sib", N_sim)
asd_vs_match = TopNAndCohesiveness(ASD_Bias, g, weighted, directed, EdgeWeightsDict, "match", N_sim)

In [ ]:
for i in range(213):
    print(i, asd_vs_rand[i]-asd_vs_rand[i-1])

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0.75,4))
plt.savefig("figs/topN_cohesivness_ratio.pdf")

In [ ]:
for i, v in enumerate(asd_vs_sib):
    print(i,v)

In [ ]:
N_sim = 1000
weighted, directed = True, True
asd_vs_rand = TopNAndCohesiveness(ASD_Bias, g, weighted, directed, EdgeWeightsDict, "rand", N_sim)
asd_vs_sib = TopNAndCohesiveness(ASD_Bias, g, weighted, directed, EdgeWeightsDict, "sib", N_sim)
asd_vs_match = TopNAndCohesiveness(ASD_Bias, g, weighted, directed, EdgeWeightsDict, "match", N_sim)

fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0.75,4))
plt.savefig("figs/topN_cohesivness_ratio.wd.pdf")

In [ ]:
adj_mat = pd.read_csv(
    "../dat/allen-mouse-conn/norm_density-max_ipsi_contra-pval_0.05-deg_min_1-by_weight_pvalue.csv",
    index_col="ROW")
str2reg = STR2Region()
ALL_STRs = adj_mat.index.values
adj_mat_local = []
adj_mat_distal = []
for str_i in ALL_STRs:
    tmp_local = []
    tmp_distal = []
    for str_j in ALL_STRs:
        weight = adj_mat.loc[str_i, str_j]
        if weight == 0:
            tmp_local.append(0)
            tmp_distal.append(0)
        else:
            rg_i = str2reg[str_i]
            rg_j = str2reg[str_j]
            if rg_i == rg_j:
                tmp_local.append(weight)
                tmp_distal.append(0)
            else:
                tmp_local.append(0)
                tmp_distal.append(weight)
    adj_mat_local.append(tmp_local)
    adj_mat_distal.append(tmp_distal)
adj_mat_local = pd.DataFrame(data=adj_mat_local, index=ALL_STRs, columns=ALL_STRs)
adj_mat_distal = pd.DataFrame(data=adj_mat_distal, index=ALL_STRs, columns=ALL_STRs)

In [ ]:
type("xxx") == str
g_local = LoadConnectome2(ConnFil=adj_mat_local)
g_distal = LoadConnectome2(ConnFil=adj_mat_distal)
EdgeWeightsDict_local = EdgeDict(g_local, keyon="label")
EdgeWeightsDict_distal = EdgeDict(g_distal, keyon="label")

In [ ]:
ASD_Cohe, Cont_Cohe = [], []
weighted, directed = False, False
for topN in range(10, 50, 1):
    CandidateNodes = ASD_Bias.head(topN).index.values
    asd = ScoreSTRSet(g_distal, CandidateNodes, EdgeWeightsDict, Weighted=weighted, Direction=directed)
    asd = np.mean(asd)
    cont_cohe = ContTopNvsCohe(ASD_Bias, topN, weighted, directed, g_distal, EdgeWeightsDict, "sib", N_sim)
    ASD_Cohe.append(asd)
    Cont_Cohe.append(cont_cohe)

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(10,50,1)), ASD_Cohe, label="ASD", color="red", linestyle="-")
ax.plot(list(range(10,50,1)), Cont_Cohe, label="sib", color="blue", linestyle="-")
ax.set_xlim(50, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")

In [ ]:
N_sim = 1000
weighted, directed = False, False
asd_vs_rand = TopNAndCohesiveness(ASD_Bias, g_distal, weighted, directed, EdgeWeightsDict_distal, "rand", N_sim)
asd_vs_sib = TopNAndCohesiveness(ASD_Bias, g_distal, weighted, directed, EdgeWeightsDict_distal, "sib", N_sim)
asd_vs_match = TopNAndCohesiveness(ASD_Bias, g_distal, weighted, directed, EdgeWeightsDict_distal, "match", N_sim)

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0,2))
plt.savefig("figs/topN_cohesivness_ratio.distal.pdf")

In [ ]:
N_sim = 1000
weighted, directed = True, False
asd_vs_rand_distal_w = TopNAndCohesiveness(ASD_Bias, g_distal, weighted, directed, EdgeWeightsDict_distal, "rand", N_sim)
asd_vs_sib_distal_w = TopNAndCohesiveness(ASD_Bias, g_distal, weighted, directed, EdgeWeightsDict_distal, "sib", N_sim)
asd_vs_match_distal_w = TopNAndCohesiveness(ASD_Bias, g_distal, weighted, directed, EdgeWeightsDict_distal, "match", N_sim)

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand_distal_w, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib_distal_w, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match_distal_w, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0,2))
plt.savefig("figs/topN_cohesivness_ratio.distal.w.pdf")

In [ ]:
N_sim = 1000
weighted, directed = False, False
asd_vs_rand = TopNAndCohesiveness(ASD_Bias, g_local, weighted, directed, EdgeWeightsDict_local, "rand", N_sim)
asd_vs_sib = TopNAndCohesiveness(ASD_Bias, g_local, weighted, directed, EdgeWeightsDict_local, "sib", N_sim)
asd_vs_match = TopNAndCohesiveness(ASD_Bias, g_local, weighted, directed, EdgeWeightsDict_local, "match", N_sim)

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0.75,2))
plt.savefig("figs/topN_cohesivness_ratio.local.pdf")

In [ ]:
Local_adjmat_cartesian = pd.read_csv("../dat/allen-mouse-conn/adj_mat_cartesian_local.csv", index_col=0)
Global_adjmat_cartesian = pd.read_csv("../dat/allen-mouse-conn/adj_mat_cartesian_distal.csv", index_col=0)
g_local_cartesian = LoadConnectome2(Local_adjmat_cartesian)
g_global_cartesian = LoadConnectome2(Global_adjmat_cartesian)

In [ ]:
N_sim = 1000
weighted, directed = False, False
asd_vs_rand_distal_w = TopNAndCohesiveness(ASD_Bias, g_global_cartesian, weighted, directed, EdgeWeightsDict_distal, "rand", N_sim)
asd_vs_sib_distal_w = TopNAndCohesiveness(ASD_Bias, g_global_cartesian, weighted, directed, EdgeWeightsDict_distal, "sib", N_sim)
asd_vs_match_distal_w = TopNAndCohesiveness(ASD_Bias, g_global_cartesian, weighted, directed, EdgeWeightsDict_distal, "match", N_sim)

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand_distal_w, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib_distal_w, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match_distal_w, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0,2))
plt.savefig("figs/topN_cohesivness_ratio.cartesian_distal.pdf")

In [ ]:
N_sim = 1000
weighted, directed = False, False
asd_vs_rand_local_b = TopNAndCohesiveness(ASD_Bias, g_local_cartesian, weighted, directed, EdgeWeightsDict_distal, "rand", N_sim)
asd_vs_sib_local_b = TopNAndCohesiveness(ASD_Bias, g_local_cartesian, weighted, directed, EdgeWeightsDict_distal, "sib", N_sim)
asd_vs_match_local_b = TopNAndCohesiveness(ASD_Bias, g_local_cartesian, weighted, directed, EdgeWeightsDict_distal, "match", N_sim)


In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), asd_vs_rand_local_b, label="ASD vs rand", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_sib_local_b, label="ASD vs sib", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), asd_vs_match_local_b, label="ASD vs match", color="red", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("Cohesiveness Ratio")
plt.ylim((0,10))
plt.savefig("figs/topN_cohesivness_ratio.cartesian_local.pdf")

# Long dist in top ranked STRS

In [ ]:
str_reg_df = pd.read_csv("dat/structure2region.map", delimiter="\t")
Regions = set(str_reg_df["REG"].values)
Reg2STRs = {}
for REG in Regions:
    Reg2STRs[REG] = str_reg_df[str_reg_df["REG"]==REG]["STR"].values

In [ ]:
adj_mat = pd.read_csv(ConnFil, index_col=0)
adj_mat_c2c, adj_mat_other = MaskCortex2Cortex(adj_mat, Reg2STRs["Isocortex"])
adj_mat_h2h, adj_mat_other_h2h = MaskCortex2Cortex(adj_mat, Reg2STRs["Hippocampal_region"])
adj_mat_local, adj_mat_distal = LocalDistal_Region()

In [ ]:
g_local = LoadConnectome2(ConnFil=adj_mat_local)
g_distal = LoadConnectome2(ConnFil=adj_mat_distal)
EdgeWeightsDict_local = EdgeDict(g_local, keyon="label")
EdgeWeightsDict_distal = EdgeDict(g_distal, keyon="label")

g_c2c = LoadConnectome2(ConnFil=adj_mat_c2c)
g_other = LoadConnectome2(ConnFil=adj_mat_other)
EdgeWeightsDict_c2c = EdgeDict(g_c2c, keyon="label")
EdgeWeightsDict_other = EdgeDict(g_other, keyon="label")

g_h2h = LoadConnectome2(ConnFil=adj_mat_h2h)
g_other_h2h = LoadConnectome2(ConnFil=adj_mat_other_h2h)
EdgeWeightsDict_c2c = EdgeDict(g_h2h, keyon="label")
EdgeWeightsDict_other = EdgeDict(g_other_h2h, keyon="label")

In [ ]:
topN = 50
weighted = True; directed=True
CandidateNodes = ASD_Bias.head(topN).index.values
asd = ScoreSTRSet(g, CandidateNodes, EdgeWeightsDict, Weighted=weighted, Direction=directed)
cont_cohe = ContTopNvsCohe(ASD_Bias, topN, weighted, directed, g, EdgeWeightsDict, "sib", N_sim)

In [ ]:
plt.figure(dpi=120)
#sns.distplot(a=asd, hist=False, color="red", label="ASD complete connectome", kde={"bw":0.00005})
plt.hist(asd, color="red", density=1, label="ASD complete connectome", alpha=0.5)
plt.axvline(x=np.mean(asd), c="red", linestyle="--")
#sns.distplot(a=cont_cohe, hist=False, color="grey", label="sib complete connectome", kde={"bw":0.00005})
plt.hist(cont_cohe.flatten(), color="grey", density=1, label="sib complete connectome", alpha=0.5)
plt.axvline(x=np.mean(cont_cohe), c="grey", linestyle="--")
plt.xlim((0,1))
plt.legend()
plt.title("Top 50 biased Structures W:{} D:{}".format(weighted, directed))
plt.show()

In [ ]:
cont_cohe = np.array(cont_cohe)

In [ ]:
GetPermutationP(np.mean(cont_cohe, axis=1), np.mean(asd))

In [ ]:
np.mean(asd)

In [ ]:
#topN = 15
CandidateNodes = ASD_Bias.head(topN).index.values
asd = ScoreSTRSet(g_other, CandidateNodes, EdgeWeightsDict, Weighted=weighted, Direction=directed)
cont_cohe = ContTopNvsCohe(ASD_Bias, topN, weighted, directed, g_other, EdgeWeightsDict, "sib", N_sim)

In [ ]:
plt.figure(dpi=120)
sns.distplot(a=asd, hist=False, color="red", label="ASD non c2c connectome", kde={"bw":0.00005})
plt.axvline(x=np.mean(asd), c="red", linestyle="--")
sns.distplot(a=cont_cohe, hist=False, color="grey", label="sib non c2c connectome", kde={"bw":0.00005})
plt.axvline(x=np.mean(cont_cohe), c="grey", linestyle="--")
plt.xlim((0,0.8))

In [ ]:
#topN = 15
CandidateNodes = ASD_Bias.head(topN).index.values
asd = ScoreSTRSet(g_distal, CandidateNodes, EdgeWeightsDict, Weighted=weighted, Direction=directed)
cont_cohe = ContTopNvsCohe(ASD_Bias, topN, weighted, directed, g_distal, EdgeWeightsDict, "sib", N_sim)

In [ ]:
plt.figure(dpi=120)
#sns.distplot(a=asd, hist=False, color="red", label="ASD complete connectome", kde={"bw":0.00005})
plt.hist(asd, color="red", density=1, label="ASD no r2r connectome", alpha=0.5)
plt.axvline(x=np.mean(asd), c="red", linestyle="--")
#sns.distplot(a=cont_cohe, hist=False, color="grey", label="sib complete connectome", kde={"bw":0.00005})
plt.hist(cont_cohe.flatten(), color="grey", density=1, label="sib no r2r connectome", alpha=0.5)
plt.axvline(x=np.mean(cont_cohe), c="grey", linestyle="--")
plt.xlim((0,1))
plt.legend()
plt.title("Top 50 biased Structures W:{} D:{}".format(weighted, directed))
plt.show()

In [ ]:
mannwhitneyu(asd, cont_cohe.flatten())

In [ ]:
#topN = 15
CandidateNodes = ASD_Bias.head(topN).index.values
asd = ScoreSTRSet(g_other_h2h, CandidateNodes, EdgeWeightsDict, Weighted=weighted, Direction=directed)
cont_cohe = ContTopNvsCohe(ASD_Bias, topN, weighted, directed, g_other_h2h, EdgeWeightsDict, "sib", N_sim)

In [ ]:
plt.figure(dpi=120)
sns.distplot(a=asd, hist=False, color="red", label="ASD non h2h connectome", kde={"bw":0.00005})
plt.axvline(x=np.mean(asd), c="red", linestyle="--")
sns.distplot(a=cont_cohe, hist=False, color="grey", label="sib non h2h connectome", kde={"bw":0.00005})
plt.axvline(x=np.mean(cont_cohe), c="grey", linestyle="--")
plt.xlim((0,0.8))

In [ ]:
def GetWeightDist(adj_mat, candidate_STRs):
    sub_mat = adj_mat.loc[candidate_STRs, candidate_STRs]
    weightes = [x for x in sub_mat.values.flatten() if x > 0]
    return weightes

In [ ]:
xx = GetWeightDist(adj_mat, ASD_Bias.head(topN).index.values)

In [ ]:
sib_weights = []
N_weights = []
for i in range(1000):
    BiasDF_cont = pd.read_csv("dat/cont.sib.bias/ASD.sib.Spec.bias.{}.csv".format(i), index_col="STR")
    CandidateNodes = BiasDF_cont.head(topN).index.values
    sib_w = GetWeightDist(adj_mat, CandidateNodes)
    sib_weights.extend(sib_w)
    N_weights.append(len(sib_w))

In [ ]:
print(np.mean(xx), np.mean(sib_weights))
print(mannwhitneyu(xx, sib_weights))
print(len(xx), np.mean(N_weights))
PlotPermutationP(N_weights, len(xx))

In [ ]:
plt.figure(dpi=120)
plt.hist(xx, color="red", density=1, label="ASD complete connectome", alpha=0.5)
plt.axvline(x=np.mean(xx), c="red", linestyle="--")
plt.hist(sib_weights, color="grey", density=1, label="sib complete connectome", alpha=0.5)
plt.axvline(x=np.mean(sib_weights), c="grey", linestyle="--")
plt.legend()
plt.title("Weight distribution for ASD and subsampled siblings")
plt.show()

In [ ]:
xx = GetWeightDist(adj_mat_distal, ASD_Bias.head(topN).index.values)
sib_weights = []
N_weights = []
for i in range(1000):
    BiasDF_cont = pd.read_csv("dat/cont.sib.bias/ASD.sib.Spec.bias.{}.csv".format(i), index_col="STR")
    CandidateNodes = BiasDF_cont.head(topN).index.values
    sib_w = GetWeightDist(adj_mat_distal, CandidateNodes)
    sib_weights.extend(sib_w)
    N_weights.append(len(sib_w))

In [ ]:
print(np.mean(xx), np.mean(sib_weights))
print(mannwhitneyu(xx, sib_weights))
print(len(xx), np.mean(N_weights))
PlotPermutationP(N_weights, len(xx))

In [ ]:
plt.figure(dpi=120)
plt.hist(xx, color="red", density=1, label="ASD no r2r connectome", alpha=0.5)
plt.axvline(x=np.mean(xx), c="red", linestyle="--")
plt.hist(sib_weights, color="grey", density=1, label="sib no r2r connectome", alpha=0.5)
plt.axvline(x=np.mean(sib_weights), c="grey", linestyle="--")
plt.legend()
plt.title("Weight distribution for ASD and subsampled siblings")
plt.show()